In [1]:
import os
import torch
from sklearn.decomposition import PCA
import numpy as np
import xml.etree.ElementTree as ET
import pandas
import math
import copy

In [2]:
input_folder = "embeddings/"
flame_folder = "base_flames/"
flame = "the_last_before_the_winter_escherymack.flame"

In [3]:
def get_embedding_filename(i, mode="tgt"):
    return os.path.join(input_folder, f"{i}.pt-{mode}_embeddings.txt")

def get_embedding_matrix(i, mode="tgt"):
    text = open(get_embedding_filename(i, mode=mode))
    lines = text.readlines()
    current_matrix = np.zeros((345, 64))
    for j, line in enumerate(lines):
        values = np.array([float(value) for value in line.split(" ")[1:]])
        current_matrix[j] = values
    return current_matrix

def RGB_format(colour_tuple):
    return ('%02x%02x%02x' % colour_tuple).upper()

def RGB_palette(number):
    df = pandas.read_csv(f"../palettes/Craig2/data/palette_{number}.csv", header=None)
    palette = [RGB_format(tuple(int(i) for i in j)) for j in df.values]
    final_string = ""
    for i, colour in enumerate(palette):
        final_string += colour
        if i % 32 == 31:
            final_string += '\n'
    return final_string

def access_coeffs(coeff_string):
    coeffs = [float(coeff) for coeff in coeff_string.split(" ")]
    return coeffs

def to_coeff_string(coeff_list):
    return " ".join([str(coeff) for coeff in coeff_list])

In [4]:
get_embedding_filename(2)

'embeddings/2.pt-tgt_embeddings.txt'

In [5]:
def get_embeddings_updates(mode="tgt"):
    old_matrix = None
    updates = np.zeros((29, 345, 64))
    for i in range(1, 30):
        current_matrix = get_embedding_matrix(i, mode=mode)
        if old_matrix is not None:
            updates[i - 2] = current_matrix - old_matrix
        old_matrix = current_matrix
    return updates

In [6]:
def build_pca(x, n, mode="tgt"):
    x = x - x.mean(axis=0)
    pca = PCA(n, whiten=False)
    pca.fit(x)
    return pca

In [7]:
def get_fractal_updates(n, mode="tgt", shuffle=True):
    epochs = get_embeddings_updates()
    reducted = np.zeros((29, 345, n))
    pca = build_pca(epochs[0], n, mode=mode)
    for i, epoch in enumerate(epochs):
        transformed = pca.transform(epoch)
        reducted[i] = transformed
    #     print(np.absolute(transformed).sum(axis=0))
    if shuffle:
        reducted = np.transpose(reducted)
        np.random.shuffle(reducted)
        reducted = np.transpose(reducted)
    return reducted

In [8]:
fractal_updates = get_fractal_updates(32)

In [9]:
print(fractal_updates.sum(axis=0).sum(axis=0))
print(np.absolute(fractal_updates).sum(axis=0).sum(axis=0))

[ -9.53280728  10.0664203  -16.32617743  -0.08721437 -25.75189574
 -25.75780754 -10.82812272   2.64352717 -12.71601173  -6.872663
  -1.8894922  -18.10184897  11.43724851  -3.04875844 -16.86999654
  -3.83240829  -3.34930837  -7.51700187   2.84788887   1.54037245
  -0.8202627   11.17497403  -0.73354152  46.13656515   2.38837873
   4.60921123  -8.87686071  -2.87351733  -0.64102047  -0.28418109
  50.69269504   4.13318777]
[228.07482477 192.71174244 208.30061028 136.45549857 176.86740959
 194.95569712 216.15426581 209.2403849  187.82350265 109.82707221
 120.47151454 222.59915859 156.47905671  97.27560588 179.55504774
  97.28526619 154.23420658 231.75556816 144.72272422 119.64825842
 120.28691703 209.65214019 111.17230983 181.9911775  122.75816702
 110.81880978 117.51538129 126.81422496 220.52890493 101.25998989
 221.83678855 130.14499165]


In [10]:
def xml_to_vector(file):
    tree = ET.parse(file)
    xforms = tree.findall("xform")
    n_xforms = len(xforms)
    param_vector = np.zeros(8 * n_xforms)
    for i, xform in enumerate(xforms):
        param_vector[i*8] = xform.attrib['weight']
        param_vector[i*8 + 1] = xform.attrib['opacity']
        param_vector[i*8+2:(i+1)*8] = np.array(access_coeffs(xform.attrib["coefs"]))
    return param_vector

In [11]:
xml_to_vector(os.path.join(flame_folder, flame))

array([ 0.5     ,  1.      ,  1.      ,  0.      ,  0.      ,  1.      ,
       -0.590569,  0.254755,  5.      ,  1.      , -1.119654, -1.068305,
        0.854836, -1.019848, -0.115798, -0.758476,  0.1     ,  0.1     ,
        1.      ,  0.      ,  0.      ,  1.      , -0.590569,  0.254755,
        0.1     ,  0.1     ,  1.      ,  0.      ,  0.      ,  1.      ,
       -0.590569,  0.254755])

In [12]:
def vector_to_xml(tree, param_vector):
    xforms = tree.findall("xform")
    n_xforms = len(xforms)
    for i, xform in enumerate(xforms):
        xform.attrib['weight'] = str(param_vector[i*8])
        xform.attrib['opacity'] = str(param_vector[i*8 + 1])
        xform.attrib["coefs"] = to_coeff_string(param_vector[i*8+2:(i+1)*8])
    return tree

In [13]:
def main(folder, file, frame_number, palette_number=None):
    
    leading_zeroes = math.ceil(math.log10(frame_number))
    
    root = ET.Element("Flames")
    root.set('name', 'flamepack')
    
    file = os.path.join(folder, file)
    tree = ET.parse(file)
    if palette_number is not None:
        tree.find("palette").text = RGB_palette(palette_number)

    
    param_vector = xml_to_vector(file)
    n = param_vector.shape[0]
    
    fractal_updates = get_fractal_updates(n)
    fractal_updates = fractal_updates[:20, 2:, :]
    fractal_updates = fractal_updates.reshape((-1, n))
    block_size = fractal_updates.shape[0] // frame_number
    fractal_updates = fractal_updates[:block_size*frame_number, :]
    
    normalization_vector = np.absolute(param_vector)
    normalization_vector[normalization_vector == 0] = 1
    fractal_total_weight = np.absolute(fractal_updates.sum(axis=0))
    
    fractal_updates = fractal_updates / fractal_total_weight * normalization_vector
    fractal_updates = np.flip(fractal_updates, axis=0)

    
    for i in range(frame_number):
        update = fractal_updates[i*block_size:(i+1)*block_size].sum(axis=0)
        param_vector = param_vector + update
        new_tree = copy.deepcopy(tree)
        new_tree = vector_to_xml(new_tree, param_vector)
        new_tree.getroot().attrib["name"] = str(frame_number - i).zfill(leading_zeroes)
        root.append(new_tree.getroot())
        
    final_tree = ET.ElementTree(root)
    final_tree.write(os.path.join(folder, "updates.flame"))

In [14]:
main("personal/crucible_highres", "crucible.flame", 1800)

In [10]:
def change_palette(file):
    
    root = ET.Element("Flames")
    root.set('name', 'flamepack')
    
    for palette_number in range(1, 31):    
        tree = ET.parse(file)
        tree.find("palette").text = RGB_palette(palette_number)
        root.append(tree.getroot())
        
    final_tree = ET.ElementTree(root)
    final_tree.write("base_flames.flame")

In [13]:
root = ET.Element("Flames")
root.set('name', 'flamepack')

tree = ET.parse("personal/hearts.flame")
tree.find("palette").text = RGB_palette(29)
root.append(tree.getroot())

final_tree = ET.ElementTree(root)
final_tree.write("hearts.flame")